In [0]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
! pip install praw --upgrade

Requirement already up-to-date: praw in /usr/local/lib/python3.6/dist-packages (6.0.0)


In [0]:
import datetime as dt
import praw # python Reddit API Wrapper

In [0]:
# set the praw module and get the acces to the reddit API

reddit = praw.Reddit(client_id='NDCve4CTpQA9sA', \
                     client_secret='L-pmEs1BwIdtNZfvNfedrxCdn-c', \
                     user_agent='mumu_parser', \
                     username='mumu_parser', \
                     password='howcanweparse2018')

In [0]:
# create a null dictionary with columns where we will store the data gathered
site_dict = {"title":[], "id":[], "url":[], "num_comments": [], "score":[]}


for treads in reddit.domain("isidewith.com").hot(limit = 2000): # parse the domain
  site_dict["title"].append(treads.title)
  site_dict["id"].append(treads.id)
  site_dict["url"].append(treads.url)
  site_dict["num_comments"].append(treads.num_comments)
  site_dict["score"].append(treads.score)

In [97]:
len(site_dict["title"])

585

Мы смогли собрать информацию о 585 сабмишенах из-за ограничения Reddit'а. Так как мы работаем не с большими данными, для простоты можем обрамить их в пандосовский датасет -- работать будет быстро. 

In [98]:
site_dict["title"][1]  # look on the first title 

'Unsure of who you are voting for tomorrow? Here is a pretty helpful quiz that matches your views with the candidates.'

In [0]:
data = pd.DataFrame(site_dict)

In [101]:
data.head()

,id,num_comments,score,title,url
0,9unyus,1,14,Everyone go out there and vote! Not sure yet? ...,https://www.isidewith.com/
1,9uerzj,6,5,Unsure of who you are voting for tomorrow? Her...,https://www.isidewith.com/michigan-voter-guide
2,9tayih,7,22,Your vote counts - find out which candidate su...,https://www.isidewith.com
3,9fb5j9,9,6,UK ISideWith Survey [2017 data],https://uk.isidewith.com/political-quiz
4,8h42bd,12,23,"""Kids will be kidnapped from good families in ...",https://www.isidewith.com/poll/965633/stance/3...


In [0]:
data.to_csv('Reddit.csv', index=False) # save data to a csv file

# Subreddit

In [0]:
subreddit = reddit.subreddit('politics')

In [124]:
for submission in subreddit.top(limit=10):
    print(submission.title, submission.id)

Kim Davis, clerk who refused to sign marriage licenses for gay couples, loses to Democrat 9uuhl3
Trump revealed highly classified information to Russian foreign minister and ambassador. 6bd42j
Trump Ordered Mueller Fired, but Backed Off When White House Counsel Threatened to Quit 7t12e6
Ivanka Trump used a personal email account to send hundreds of emails about government business last year 9ymbzc
A petition calling for FCC Chairman Ajit Pai to resign is racking up huge numbers 7fpqwf
The FCC’s craven net neutrality vote announcement makes no mention of the 22 million comments filed 7ekkmo
'A Red Line Crossed': Nationwide Protests Declared for Thursday at 5PM After Jeff Sessions Fired 9v57rh
Alexandria Ocasio-Cortez Says Lawmakers Should Lose Their Salaries Too When They Shut Down the Government a8nmov
The FCC has unveiled its plan to repeal its net neutrality rules 7eizpg
‘Cards Against Humanity’ Creator Just Pledged To Buy and Publish Congress’s Browser History 62a3kj


In [0]:
# create a null dictionary with columns where we will store the data gathered
subred_dict = {"title":[], "id":[], "url":[], "num_comments": [], "score":[], "body":[]}

for submission in subreddit.top(limit = 2000): # parse the domain
  subred_dict["title"].append(submission.title)
  subred_dict["id"].append(submission.id)
  subred_dict["url"].append(submission.url)
  subred_dict["num_comments"].append(submission.num_comments)
  subred_dict["score"].append(submission.score)
  subred_dict["body"].append(submission.selftext)

In [0]:
df = pd.DataFrame(subred_dict)

In [128]:
len(df["id"]) # 998 is due to the limitations of Reddit (< 1000)

998

In [0]:
sub_comments = []

for comment in subreddit.comments(limit=2000):
    sub_comments.append(comment.body)

In [134]:
sub_comments[10]

'Big difference between my mother and myself. She once punished me for not standing up to a bully when they were messing with my friend citing guilt by association. I took it to heart... She taught me something that she refuses to take on herself. '

In [0]:
from IPython import display
from pprint import pprint
import nltk
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [138]:
! pip install nltk --upgrade

    100% |████████████████████████████████| 1.4MB 15.1MB/s 
  Running setup.py bdist_wheel for nltk ... - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [140]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d)
Command 'd)' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Sys

True

In [142]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for comment in sub_comments:
    pol_score = sia.polarity_scores(comment)
    pol_score['sub_comments'] = comment
    results.append(pol_score)

pprint(results[:3], width=100)

[{'compound': 0.0,
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'sub_comments': 'something about '
                  '[`https://mobile.abc.net.au/news/image/10669502-3x2-700x467.jpg`](https://mobile.abc.net.au/news/image/10669502-3x2-700x467.jpg) '
                  "this image doesn't look right."},
 {'compound': -0.661,
  'neg': 0.389,
  'neu': 0.494,
  'pos': 0.117,
  'sub_comments': '>Hmm, you have chosen to argue that "death isn\'t death"?\n\nNo\n\n'},
 {'compound': 0.0,
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'sub_comments': 'There was actually more volume today than most other days this year.'}]


In [143]:
sent = pd.DataFrame.from_records(results)
sent.head()

,compound,neg,neu,pos,sub_comments
0,0.0000,0.000,1.000,0.000,something about [`https://mobile.abc.net.au/ne...
1,-0.6610,0.389,0.494,0.117,">Hmm, you have chosen to argue that ""death isn..."
2,0.0000,0.000,1.000,0.000,There was actually more volume today than most...
3,0.7425,0.073,0.706,0.221,Republicans and Democrats had bipartisan polic...
4,-0.5719,0.147,0.749,0.105,The problem is that ultimately its the POTUS t...


In [152]:
np.mean(sent['compound'])

-0.016487168610816543

In [153]:
np.min(sent['compound'])

-0.9987

In [154]:
np.max(sent['compound'])

0.973

In [155]:
sent['label'] = 0
sent.loc[sent['compound'] > 0.3, 'label'] = 1
sent.loc[sent['compound'] < -0.3, 'label'] = -1
sent.head()

,compound,neg,neu,pos,sub_comments,label
0,0.0000,0.000,1.000,0.000,something about [`https://mobile.abc.net.au/ne...,0
1,-0.6610,0.389,0.494,0.117,">Hmm, you have chosen to argue that ""death isn...",-1
2,0.0000,0.000,1.000,0.000,There was actually more volume today than most...,0
3,0.7425,0.073,0.706,0.221,Republicans and Democrats had bipartisan polic...,1
4,-0.5719,0.147,0.749,0.105,The problem is that ultimately its the POTUS t...,-1


In [0]:
# save to a csv file 
huh = sent[['sub_comments', 'label']]
huh.to_csv('reddit_headlines_labels.csv', mode='a', encoding='utf-8', index=False)

In [158]:
print("Positive comments:\n")
pprint(list(sent[sent['label'] == 1].sub_comments)[:5], width=200)

print("\nNegative commments:\n")
pprint(list(sent[sent['label'] == -1].sub_comments)[:5], width=200)

Positive comments:

['Republicans and Democrats had bipartisan policy to pass in 2016 for 2017 so that our government could more easily control our narrative.  Both sides needed a boogie man and "fake news" fit the '
 'bill perfectly.  ',
 "Really? We need an article to validate it? Really?\nShouldn't this be obvious?",
 "> It's a lack of critical thinking being taught in schools and universities thanks to a massive reduction in funding for humanities subjects.\n"
 '\n'
 'Eh, it\'s a bit more than that.  The "team first" mentality is pervasive.  I see it exhibited even among folks I respect and who I view as sublimely critical thinkers.  \n'
 "There's an emotional response that some nurture in themselves.  Really taken all of the fun out of political discourse.  Especially on the Internet.  ",
 "Dave Sirota wanted O'Rourke as Speaker of the House, AOC as whip, etc. Now he does nothing but tweet about Beto and whine about how he's just a freedom fighter who is obligated by duty to expos

In [162]:
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r'\w+')


stop_words = stopwords.words('english')
print(stop_words[:20])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [0]:
def process_text(sub_comments):
    tokens = []
    for comment in sub_comments:
        toks = tokenizer.tokenize(comment)
        toks = [t.lower() for t in toks if t.lower() not in stop_words]
        tokens.extend(toks)
    
    return tokens

**Positive words**

In [163]:
pos_lines = list(sent[sent.label == 1].sub_comments)

pos_tokens = process_text(pos_lines)
pos_freq = nltk.FreqDist(pos_tokens)

pos_freq.most_common(20)

[('like', 64),
 ('trump', 58),
 ('people', 52),
 ('politics', 32),
 ('would', 30),
 ('good', 26),
 ('one', 26),
 ('time', 26),
 ('r', 25),
 ('even', 24),
 ('https', 24),
 ('much', 24),
 ('think', 24),
 ('com', 23),
 ('get', 21),
 ('make', 21),
 ('know', 20),
 ('www', 18),
 ('better', 18),
 ('point', 18)]

**Negative words**

In [165]:
neg_lines = list(sent[sent.label == -1].sub_comments)

neg_tokens = process_text(neg_lines)
neg_freq = nltk.FreqDist(neg_tokens)

neg_freq.most_common(20)

[('trump', 91),
 ('people', 60),
 ('would', 56),
 ('like', 47),
 ('right', 34),
 ('politics', 31),
 ('shit', 31),
 ('even', 30),
 ('r', 30),
 ('want', 30),
 ('see', 29),
 ('us', 29),
 ('get', 28),
 ('death', 27),
 ('news', 27),
 ('one', 26),
 ('know', 25),
 ('think', 25),
 ('still', 24),
 ('war', 24)]

**Building a fuction**

In [0]:
# import praw

def reddit_parser(subreddit_name, NUM = 1000):
  # returns a dataset with general info
  
  # set the praw module and get the acces to the reddit API
  reddit = praw.Reddit(client_id='NDCve4CTpQA9sA', \
                       client_secret='L-pmEs1BwIdtNZfvNfedrxCdn-c', \
                       user_agent='mumu_parser', \
                       username='mumu_parser', \
                       password='howcanweparse2018')

  subreddit = reddit.subreddit(subreddit_name) # your subreddit name

  ### general subreddit section ###
  # create a null dictionary with columns where we will store the data gathered
  subred_dict = {"title":[], "id":[], "url":[], "num_comments": [], "score":[], "body":[]}

  for submission in subreddit.top(limit = NUM): # parse info on top posts in your subreddit 
    subred_dict["title"].append(submission.title)
    subred_dict["id"].append(submission.id)
    subred_dict["url"].append(submission.url)
    subred_dict["num_comments"].append(submission.num_comments)
    subred_dict["score"].append(submission.score)
    subred_dict["body"].append(submission.selftext)


  return pd.DataFrame(subred_dict)

In [0]:
# import nltk

def redditcomments_parser(subreddit_name, NUM = 1000):
  # returns a dataset with comments of a subreddit section
  
  # set the praw module and get the acces to the reddit API
  reddit = praw.Reddit(client_id='NDCve4CTpQA9sA', \
                       client_secret='L-pmEs1BwIdtNZfvNfedrxCdn-c', \
                       user_agent='mumu_parser', \
                       username='mumu_parser', \
                       password='howcanweparse2018')

  subreddit = reddit.subreddit(subreddit_name) # your subreddit name

  ### comments section ###
  sub_comments = []

  for comment in subreddit.comments(limit=NUM):
      sub_comments.append(comment.body)

  from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA # first download all the packages

  sia = SIA()
  results = []

  for comment in sub_comments:
      pol_score = sia.polarity_scores(comment)
      pol_score['sub_comments'] = comment
      results.append(pol_score)

  return pd.DataFrame.from_records(results)

In [176]:
reddit_parser('politics', NUM=10)

,body,id,num_comments,score,title,url
0,,9uuhl3,2777,100731,"Kim Davis, clerk who refused to sign marriage ...",https://www.kentucky.com/news/politics-governm...
1,,6bd42j,20906,99354,Trump revealed highly classified information t...,https://www.washingtonpost.com/world/national-...
2,,7t12e6,14604,95195,"Trump Ordered Mueller Fired, but Backed Off Wh...",https://www.nytimes.com/2018/01/25/us/politics...
3,,9ymbzc,6615,89273,Ivanka Trump used a personal email account to ...,https://www.washingtonpost.com/politics/ivanka...
4,,7fpqwf,1954,88282,A petition calling for FCC Chairman Ajit Pai t...,https://www.dailydot.com/layer8/we-the-petitio...
5,,7ekkmo,1728,87633,The FCC’s craven net neutrality vote announcem...,https://techcrunch.com/2017/11/21/the-fccs-cra...
6,,9v57rh,8442,82702,'A Red Line Crossed': Nationwide Protests Decl...,https://www.commondreams.org/news/2018/11/07/r...
7,,a8nmov,4116,82393,Alexandria Ocasio-Cortez Says Lawmakers Should...,https://www.newsweek.com/alexandria-ocasio-cor...
8,,7eizpg,5785,81983,The FCC has unveiled its plan to repeal its ne...,https://www.washingtonpost.com/news/the-switch...
9,,62a3kj,2359,80378,‘Cards Against Humanity’ Creator Just Pledged ...,http://resistancereport.com/news/cards-humanit...


In [177]:
redditcomments_parser("politics", NUM=10)

,compound,neg,neu,pos,sub_comments
0,-0.7543,0.194,0.806,0.000,Im sorry you have trouble seeing through the e...
1,-0.9619,0.225,0.746,0.028,"I'm trying to avoid mislabeling it, nothing mo..."
2,-0.8074,0.274,0.726,0.000,So he can't sit down with someone who he has n...
3,0.8349,0.000,0.759,0.241,Some of the Bernie people were running Beto's ...
4,-0.9432,0.192,0.766,0.042,Here's the problem with this comparison. Liter...
5,-0.5367,0.078,0.922,0.000,Trump is doing this so he can steal 5bil from ...
6,0.0000,0.000,1.000,0.000,He just went for the headlines and photo op. ...
7,0.5230,0.120,0.732,0.147,>A spokeswoman for the College Sports Informat...
8,0.6808,0.090,0.711,0.199,"Ffs, just run the best candidate. Period. Man,..."
9,-0.7981,0.223,0.723,0.054,"Dude, I don't like him and have put him down a..."
